In [1]:
import numpy as np
import cv2
import glob


# Load previously saved data
with np.load('CameraParams.npz') as file:
    mtx, dist, rvecs, tvecs = [file[i] for i in ('cameraMatrix','dist','rvecs','tvecs')]

In [2]:
def cameraPoseFromHomography(H):  
    norm1 = np.linalg.norm(H[:, 0])
    norm2 = np.linalg.norm(H[:, 1])
    tnorm = (norm1 + norm2) / 2.0

    H1 = H[:, 0] / norm1
    H2 = H[:, 1] / norm2
    H3 = np.cross(H1, H2)
    T = H[:, 2] / tnorm

    return np.array([H1, H2, H3, T]).transpose()

In [3]:
import numpy as np 
import cv2 
  
# Завантаження зображень 
image1 = cv2.imread('objImages/1.jpg')  
image1 = cv2.resize(image1, (480, 480))
image2 = cv2.imread('objImages/5.jpg') 
image2 = cv2.resize(image2, (620, 480))

sift = cv2.SIFT_create() 

  
keypoints1, descriptors1 = sift.detectAndCompute(image1, None) 
keypoints2, descriptors2 = sift.detectAndCompute(image2, None) 

BFMatch = cv2.BFMatcher(cv2.NORM_L2) 
Matches = BFMatch.knnMatch(descriptors1, descriptors2, k=2) 

good_matches = []
for m, n in Matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)


src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

result = cameraPoseFromHomography(H)
R = result[:, :-1]
t = result[:, -1]
print(R, t)
R, _ = cv2.Rodrigues(R)
print(R, t)

matchesMask = mask.ravel().tolist()
h,w = image1.shape[:2]
pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)

[[-9.31206192e-01 -9.92519265e-01  5.40494139e-04]
 [ 3.64488487e-01  1.22084899e-01  2.59285044e-03]
 [-1.78071760e-03  8.86435164e-04  2.48075631e-01]] [1083.38624044  390.63159135    2.28935262]
[[-0.00396169]
 [ 0.01232281]
 [ 2.10843145]] [1083.38624044  390.63159135    2.28935262]


In [4]:
def my_estimatePoseSingleMarkers(corners, marker_size, mtx, distortion):
    marker_points = np.array([[-marker_size / 2, marker_size / 2, 0],
                              [marker_size / 2, marker_size / 2, 0],
                              [marker_size / 2, -marker_size / 2, 0],
                              [-marker_size / 2, -marker_size / 2, 0]], dtype=np.float32)

    nada, R, t = cv2.solvePnP(marker_points, corners, mtx, distortion, False, cv2.SOLVEPNP_IPPE_SQUARE)

    return np.array(R), np.array(t), nada

In [5]:
corners = cv2.perspectiveTransform(pts, H)[::-1]

rvec, tvec, markerPoints = my_estimatePoseSingleMarkers(corners, 0.02, mtx, dist)

img3 = image2
img3 = cv2.drawFrameAxes(img3, mtx, dist, rvec, tvec, 0.01)  

for corner in corners:
        x, y = corner[0]
        frame = cv2.circle(img3, (int(x), int(y)), 5, (0, 0, 255), -1) 

img3 = cv2.resize(img3, (600, 600))
cv2.imshow("result", img3)
cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
def pose_estimation(frame, mtx, dist, sift, keypoints1, descriptors1):
    grey_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    keypoints2, descriptors2 = sift.detectAndCompute(grey_frame, None) 
    
    try:
        BFMatch = cv2.BFMatcher() 
        Matches = BFMatch.knnMatch(descriptors1, descriptors2, k=2) 

        # Фільтрація матчів за допомогою ratio test 
        good_matches = []
        for m, n in Matches:
            if m.distance < 0.7 * n.distance:
                good_matches.append(m)


        src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        h,w = image1.shape[:2]
        pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)

        corners = cv2.perspectiveTransform(pts, H)
    except:
        return frame

    corners = corners[::-1]
    rvec, tvec, markerPoints = my_estimatePoseSingleMarkers(corners, 0.02, mtx, dist)

    frame = cv2.drawFrameAxes(frame, mtx, dist, rvec, tvec, 0.01)  

    for corner in corners:
        x, y = corner[0]
        frame = cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1) 
    
    return frame


In [9]:
cap = cv2.VideoCapture(1)

sift = cv2.SIFT_create() 

image1 = cv2.imread('objImages/1.jpg')  
image1 = cv2.resize(image1, (480, 480))
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
  
keypoints1, descriptors1 = sift.detectAndCompute(image1, None) 

while cap.isOpened():
    
    ret, img = cap.read()

    timer = cv2.getTickCount()
    
    frame = pose_estimation(img, mtx, dist, sift, keypoints1, descriptors1)

    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

    frame = cv2.putText(frame, f'{int(fps)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (50, 170, 50), 3)
    cv2.imshow('Estimated Pose', frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()